In [198]:
from helpers import *

import sys

import bz2
import json

import numpy as np
import scipy

import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.pipeline import Pipeline

from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.neural_network import MLPClassifier

from sklearn.model_selection import cross_val_score, cross_validate

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [199]:
# XGBoost

from xgboost import XGBClassifier

In [200]:
# Printing without trunctions
# np.set_printoptions(threshold=sys.maxsize)
pd.set_option('display.max_colwidth', None)

In [201]:
path = fixpath(QUOTES_2020_LABELED_CLEANED_VARIANTS_MINI)
# path = fixpath(QUOTES_2020_LABELED_CLEANED_VARIANTS)

df_raw = pd.read_json(path, orient='records', lines=True)
df_raw

,quoteID,quotation,speaker,date,numOccurrences,id,party_label,US_congress_bio_ID,quotation_cleanA,quotation_cleanB,quotation_cleanC,quotation_cleanD,quotation_cleanE
0,2020-01-16-000088,[ Department of Homeland Security ] was livid and strongly urged to have the agenda pulled.,sue myrick,2020-01-16 12:00:13,1,Q367796,R,M001134,[ Department of Homeland Security ] was livid and strongly urged to have the agenda pulled.,department of homeland security was livid and strongly urged to have the agenda pulled,department homeland security livid strongly urged agenda pulled,depart homeland secur livid strong urg agenda pull,department homeland security livid strongly urged agenda pulled
1,2020-01-26-000499,a few of the candidates who will do better in our part of the world,dave loebsack,2020-01-26 13:21:36,11,Q771586,D,L000565,a few of the candidates who will do better in our part of the world,a few of the candidates who will do better in our part of the world,candidates better part world,candid better part world,candidate better part world
2,2020-01-26-040663,"The generational thing is important, quite honestly. I think we have to do everything we can to move this party forward and Pete is the future of the party,",dave loebsack,2020-01-26 13:21:36,11,Q771586,D,L000565,"The generational thing is important, quite honestly. I think we have to do everything we can to move this party forward and Pete is the future of the party,",the generational thing is important quite honestly i think we have to do everything we can to move this party forward and pete is the future of the party,generational thing important quite honestly think everything move party forward pete future party,generat thing import quit honest think everyth move parti forward pete futur parti,generational thing important quite honestly think everything move party forward pete future party
3,2020-01-20-000982,a host of other protections,debbie lesko,2020-01-20 15:32:48,1,Q16731415,R,L000589,a host of other protections,a host of other protections,host protections,host protect,host protection
4,2020-01-09-060095,"No, are you kidding? The way that this place leaks around here?",debbie lesko,2020-01-09 23:15:21,1,Q16731415,R,L000589,"No, are you kidding? The way that this place leaks around here?",no are you kidding the way that this place leaks around here,kidding way place leaks around,kid way place leak around,kidding way place leak around
...,...,...,...,...,...,...,...,...,...,...,...,...,...
349141,2020-02-10-100845,"We're just worried about making sure we keep that balance between expression of our rights and public safety,",sherrie sprenger,2020-02-10 00:00:00,1,Q7495360,R,None,"We're just worried about making sure we keep that balance between expression of our rights and public safety,",we re just worried about making sure we keep that balance between expression of our rights and public safety,worried making sure keep balance expression rights public safety,worri make sure keep balanc express right public safeti,worried making sure keep balance expression right public safety
349142,2020-03-16-079753,"We've all embraced strict proper hygiene procedures that all of you by now have heard a hundred times. But believe me, they work,",robert abrams,2020-03-16 12:00:00,2,Q2156314,D,None,"We've all embraced strict proper hygiene procedures that all of you by now have heard a hundred times. But believe me, they work,",we ve all embraced strict proper hygiene procedures that all of you by now have heard a hundred times but believe me they work,embraced strict proper hygiene procedures heard hundred times believe work,embrac strict proper hygien procedur heard hundr time believ work,embraced strict proper hygiene procedure heard hundred time believe work
349143,2020-01-13-091997,What's important is that we keep moving forward.,laurie jinkins,2020-01-13 19:51:15,1,Q6501617,D,None,What's important is that we keep moving forward.,what s important is that we keep moving forward,import

In [202]:
df = df_raw.copy()

In [203]:
lengths_of_df = df['quotation_cleanE'].apply(lambda x: len(x))
lengths_of_df.median()

64.0

In [204]:
# df = df[np.logical_and(lengths_of_df > lengths_of_df.quantile(0.3), lengths_of_df < lengths_of_df.quantile(0.9))]
f = df[lengths_of_df > lengths_of_df.quantile(0.2)]
df.shape

(349146, 13)

In [205]:
df['party_label'].value_counts()

D    218894
R    130252
Name: party_label, dtype: int64

In [206]:
def downsample(df:pd.DataFrame, label_col_name:str, force_sample_n=None) -> pd.DataFrame:
    # find the number of observations in the smallest group

    if force_sample_n != None:
        nmin = force_sample_n
    else:
        nmin = df[label_col_name].value_counts().min()
    return (df
            # split the dataframe per group
            .groupby(label_col_name)
            # sample nmin observations from each group
            .apply(lambda x: x.sample(nmin))
            # recombine the dataframes
            .reset_index(drop=True)
            )

df = downsample(df, 'party_label')
# df = downsample(df, 'party_label', force_sample_n=5000)

In [207]:
df['party_label'].value_counts()

D    130252
R    130252
Name: party_label, dtype: int64

### Example usage of sklean Model

In [148]:
X = TfidfVectorizer().fit_transform(df['quotation_cleanD'].values)
y = df['party_label'].values
y = np.array([0 if label=='R' else 1 for label in y])
y, X

(array([1, 1, 1, ..., 0, 0, 0]),
 <191516x28839 sparse matrix of type '<class 'numpy.float64'>'
 	with 2464815 stored elements in Compressed Sparse Row format>)

In [149]:
clf = MultinomialNB()
# clf.fit(X, y)
scores = cross_val_score(clf, X, y, cv=5)

print(f'Avg: {scores.mean():.3f}\tStd: {scores.std():.3f}')

Avg: 0.671	Std: 0.003


In [150]:
clf = MultinomialNB()

scoring=['accuracy', 'precision', 'recall', 'f1']
res = cross_validate(clf, X, y, scoring=scoring, cv=3)
{item: (value.mean(), value.std()) for (item, value) in res.items()}

{'fit_time': (0.04200975100199381, 0.007753792517962397),
 'score_time': (0.05926903088887533, 0.004610906770016475),
 'test_accuracy': (0.6679389794266553, 0.0019415389637946878),
 'test_precision': (0.6683144656132046, 0.0022751909329718246),
 'test_recall': (0.6668372176826142, 0.0013386032230737837),
 'test_f1': (0.6675739714119876, 0.001665980070333526)}

### Appling model to quotation variants

In [39]:
# import sklearn
# sorted(sklearn.metrics.SCORERS.keys())

In [208]:
cols = [
    'quotation_cleanE',
    'quotation_cleanA', 
    'quotation_cleanB',
    'quotation_cleanC',
    'quotation_cleanD',
    ]


def test_classifer(pipeline, break_after_one_iter=False):
    for col in cols:

        # X = vectorizer.fit_transform(df[col].values)
        X = df[col].values

        y = df['party_label'].values
        y = np.array([0.0 if label=='R' else 1.0 for label in y])

        # scores = cross_val_score(clf, X, y, cv=3)
        # print(f'Col: {col}\tAvg: {scores.mean():.3f}\tStd: {scores.std():.3f}')

        # scoring=['accuracy']
        # scoring=['f1']
        
        

        scoring=['accuracy', 'precision', 'recall', 'f1']
        res = cross_validate(pipeline, X, y, scoring=scoring, cv=3)
        res = {item: (value.mean(), value.std()) for (item, value) in res.items()}

        # Code isn't pretty but prints nice output!
        print(f'Col: {col}')

        print(f'X shape: {X.shape}')

        print(f'\tfit_time - \t\tavg:{res["fit_time"][0]:.3f}\tstd:{res["fit_time"][1]:.3f}')
        print(f'\tscore_time - \t\tavg:{res["score_time"][0]:.3f}\tstd:{res["score_time"][1]:.3f}')
        print(f'\ttest_accuracy - \tavg:{res["test_accuracy"][0]:.3f}\tstd:{res["test_accuracy"][1]:.3f}')
        print(f'\ttest_precision - \tavg:{res["test_precision"][0]:.3f}\tstd:{res["test_precision"][1]:.3f}')
        print(f'\ttest_recall - \t\tavg:{res["test_recall"][0]:.3f}\tstd:{res["test_recall"][1]:.3f}')
        print(f'\ttest_f1 - \t\tavg:{res["test_f1"][0]:.3f}\tstd:{res["test_f1"][1]:.3f}')
        
        if break_after_one_iter:
            break

        
    # break

In [209]:
pipeline = Pipeline([
            # ('vect', TfidfVectorizer(ngram_range=(1,1), max_features=10)),
            ('vect', TfidfVectorizer(ngram_range=(1,3), max_features=1_000_000)),
            ('clf', MultinomialNB()),
        ])

test_classifer(pipeline, break_after_one_iter=True)

Col: quotation_cleanE
X shape: (260504,)
	fit_time - 		avg:20.802	std:0.209
	score_time - 		avg:3.887	std:0.089
	test_accuracy - 	avg:0.706	std:0.002
	test_precision - 	avg:0.710	std:0.001
	test_recall - 		avg:0.697	std:0.003
	test_f1 - 		avg:0.703	std:0.002


In [25]:
pipeline = Pipeline([
            ('vect', TfidfVectorizer(ngram_range=(1,2), max_features=50000)),
            ('clf', MultinomialNB()),
        ])

test_classifer(pipeline)

Col: quotation_cleanA
X shape: (10000,)
	fit_time - 		avg:0.546	std:0.039
	score_time - 		avg:0.165	std:0.004
	test_accuracy - 	avg:0.599	std:0.011
	test_precision - 	avg:0.599	std:0.012
	test_recall - 		avg:0.596	std:0.012
	test_f1 - 		avg:0.597	std:0.011
Col: quotation_cleanB
X shape: (10000,)
	fit_time - 		avg:0.587	std:0.025
	score_time - 		avg:0.166	std:0.004
	test_accuracy - 	avg:0.599	std:0.011
	test_precision - 	avg:0.599	std:0.012
	test_recall - 		avg:0.596	std:0.012
	test_f1 - 		avg:0.597	std:0.011
Col: quotation_cleanC
X shape: (10000,)
	fit_time - 		avg:0.371	std:0.008
	score_time - 		avg:0.090	std:0.001
	test_accuracy - 	avg:0.601	std:0.004
	test_precision - 	avg:0.599	std:0.004
	test_recall - 		avg:0.611	std:0.008
	test_f1 - 		avg:0.605	std:0.005
Col: quotation_cleanD
X shape: (10000,)
	fit_time - 		avg:0.387	std:0.077
	score_time - 		avg:0.104	std:0.018
	test_accuracy - 	avg:0.606	std:0.004
	test_precision - 	avg:0.607	std:0.004
	test_recall - 		avg:0.600	std:0.013
	test

In [27]:
pipeline = Pipeline([
            ('vect', TfidfVectorizer(ngram_range=(1,2), max_features=50000)),
            ('clf', LogisticRegression(n_jobs=-1, C=1e5)),
        ])

test_classifer(pipeline)

/Users/mroust/anaconda3/envs/ada/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/mroust/anaconda3/envs/ada/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_mode

Col: quotation_cleanA
X shape: (10000,)
	fit_time - 		avg:2.623	std:0.708
	score_time - 		avg:0.196	std:0.027
	test_accuracy - 	avg:0.587	std:0.008
	test_precision - 	avg:0.588	std:0.009
	test_recall - 		avg:0.580	std:0.008
	test_f1 - 		avg:0.584	std:0.008


/Users/mroust/anaconda3/envs/ada/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/mroust/anaconda3/envs/ada/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_mode

Col: quotation_cleanB
X shape: (10000,)
	fit_time - 		avg:1.735	std:0.261
	score_time - 		avg:0.184	std:0.024
	test_accuracy - 	avg:0.587	std:0.008
	test_precision - 	avg:0.588	std:0.009
	test_recall - 		avg:0.580	std:0.008
	test_f1 - 		avg:0.584	std:0.008


/Users/mroust/anaconda3/envs/ada/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/mroust/anaconda3/envs/ada/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_mode

...

/Users/mroust/anaconda3/envs/ada/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/mroust/anaconda3/envs/ada/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_mode

Col: quotation_cleanD
X shape: (10000,)
	fit_time - 		avg:1.305	std:0.299
	score_time - 		avg:0.092	std:0.005
	test_accuracy - 	avg:0.592	std:0.009
	test_precision - 	avg:0.593	std:0.009
	test_recall - 		avg:0.586	std:0.015
	test_f1 - 		avg:0.589	std:0.012


/Users/mroust/anaconda3/envs/ada/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/mroust/anaconda3/envs/ada/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_mode

Col: quotation_cleanE
X shape: (10000,)
	fit_time - 		avg:1.503	std:0.095
	score_time - 		avg:0.096	std:0.006
	test_accuracy - 	avg:0.594	std:0.004
	test_precision - 	avg:0.594	std:0.005
	test_recall - 		avg:0.591	std:0.002
	test_f1 - 		avg:0.593	std:0.002


/Users/mroust/anaconda3/envs/ada/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [28]:
pipeline = Pipeline([
            ('vect', TfidfVectorizer(ngram_range=(1,2), max_features=100)),
            ('clf', RandomForestClassifier(n_estimators=10, n_jobs=7)),
        ])

test_classifer(pipeline)

Col: quotation_cleanA
X shape: (10000,)
	fit_time - 		avg:0.987	std:0.309
	score_time - 		avg:0.150	std:0.009
	test_accuracy - 	avg:0.511	std:0.012
	test_precision - 	avg:0.514	std:0.015
	test_recall - 		avg:0.410	std:0.012
	test_f1 - 		avg:0.456	std:0.013
Col: quotation_cleanB
X shape: (10000,)
	fit_time - 		avg:0.758	std:0.004
	score_time - 		avg:0.142	std:0.005
	test_accuracy - 	avg:0.506	std:0.007
	test_precision - 	avg:0.508	std:0.009
	test_recall - 		avg:0.413	std:0.010
	test_f1 - 		avg:0.455	std:0.005
Col: quotation_cleanC
X shape: (10000,)
	fit_time - 		avg:0.459	std:0.031
	score_time - 		avg:0.083	std:0.001
	test_accuracy - 	avg:0.507	std:0.005
	test_precision - 	avg:0.507	std:0.005
	test_recall - 		avg:0.537	std:0.073
	test_f1 - 		avg:0.519	std:0.035
Col: quotation_cleanD
X shape: (10000,)
	fit_time - 		avg:0.490	std:0.072
	score_time - 		avg:0.088	std:0.012
	test_accuracy - 	avg:0.515	std:0.007
	test_precision - 	avg:0.518	std:0.008
	test_recall - 		avg:0.429	std:0.014
	test

In [ ]:
# clf.fit(X[100:], y[100:])
# clf.predict_proba(X[0:5])

In [29]:
pipeline = Pipeline([
            ('vect', TfidfVectorizer(ngram_range=(1,2), max_features=100)),
            ('clf', GradientBoostingClassifier(n_estimators=10, learning_rate=1.0,
                                               max_depth=1, random_state=0)),
        ])

test_classifer(pipeline)

Col: quotation_cleanA
X shape: (10000,)
	fit_time - 		avg:0.752	std:0.092
	score_time - 		avg:0.140	std:0.022
	test_accuracy - 	avg:0.531	std:0.002
	test_precision - 	avg:0.530	std:0.002
	test_recall - 		avg:0.560	std:0.023
	test_f1 - 		avg:0.544	std:0.011
Col: quotation_cleanB
X shape: (10000,)
	fit_time - 		avg:0.763	std:0.118
	score_time - 		avg:0.152	std:0.023
	test_accuracy - 	avg:0.531	std:0.002
	test_precision - 	avg:0.530	std:0.002
	test_recall - 		avg:0.560	std:0.023
	test_f1 - 		avg:0.544	std:0.011
Col: quotation_cleanC
X shape: (10000,)
	fit_time - 		avg:0.414	std:0.026
	score_time - 		avg:0.076	std:0.000
	test_accuracy - 	avg:0.515	std:0.001
	test_precision - 	avg:0.540	std:0.045
	test_recall - 		avg:0.648	std:0.396
	test_f1 - 		avg:0.489	std:0.237
Col: quotation_cleanD
X shape: (10000,)
	fit_time - 		avg:0.396	std:0.007
	score_time - 		avg:0.084	std:0.011
	test_accuracy - 	avg:0.516	std:0.004
	test_precision - 	avg:0.529	std:0.025
	test_recall - 		avg:0.621	std:0.369
	test

In [30]:
pipeline = Pipeline([
            ('vect', TfidfVectorizer(ngram_range=(1,2), max_features=100)),
            ('clf', MLPClassifier(alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)),
        ])

test_classifer(pipeline)

/Users/mroust/anaconda3/envs/ada/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/mroust/anaconda3/envs/ada/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/mroust/anaconda3/envs/ada/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Col: quotation_cleanA
X shape: (10000,)
	fit_time - 		avg:7.049	std:0.961
	score_time - 		avg:0.134	std:0.010
	test_accuracy - 	avg:0.521	std:0.005
	test_precision - 	avg:0.544	std:0.009
	test_recall - 		avg:0.257	std:0.015
	test_f1 - 		avg:0.349	std:0.015


/Users/mroust/anaconda3/envs/ada/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/mroust/anaconda3/envs/ada/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/mroust/anaconda3/envs/ada/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Col: quotation_cleanB
X shape: (10000,)
	fit_time - 		avg:5.650	std:0.035
	score_time - 		avg:0.124	std:0.005
	test_accuracy - 	avg:0.521	std:0.005
	test_precision - 	avg:0.544	std:0.009
	test_recall - 		avg:0.257	std:0.015
	test_f1 - 		avg:0.349	std:0.015


/Users/mroust/anaconda3/envs/ada/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/mroust/anaconda3/envs/ada/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/mroust/anaconda3/envs/ada/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


...

/Users/mroust/anaconda3/envs/ada/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/mroust/anaconda3/envs/ada/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/mroust/anaconda3/envs/ada/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Col: quotation_cleanD
X shape: (10000,)
	fit_time - 		avg:6.383	std:0.262
	score_time - 		avg:0.080	std:0.002
	test_accuracy - 	avg:0.515	std:0.002
	test_precision - 	avg:0.530	std:0.004
	test_recall - 		avg:0.274	std:0.020
	test_f1 - 		avg:0.361	std:0.017


/Users/mroust/anaconda3/envs/ada/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/mroust/anaconda3/envs/ada/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Col: quotation_cleanE
X shape: (10000,)
	fit_time - 		avg:6.074	std:0.147
	score_time - 		avg:0.078	std:0.003
	test_accuracy - 	avg:0.513	std:0.004
	test_precision - 	avg:0.525	std:0.006
	test_recall - 		avg:0.268	std:0.013
	test_f1 - 		avg:0.355	std:0.013


/Users/mroust/anaconda3/envs/ada/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [32]:
pipeline = Pipeline([
            ('vect', TfidfVectorizer(ngram_range=(1,2), max_features=100)),
            ('clf', SGDClassifier(loss="hinge", penalty="l2", max_iter=50)),
        ])

test_classifer(pipeline)

Col: quotation_cleanA
X shape: (10000,)
	fit_time - 		avg:0.631	std:0.010
	score_time - 		avg:0.157	std:0.026
	test_accuracy - 	avg:0.535	std:0.002
	test_precision - 	avg:0.536	std:0.003
	test_recall - 		avg:0.527	std:0.025
	test_f1 - 		avg:0.531	std:0.012
Col: quotation_cleanB
X shape: (10000,)
	fit_time - 		avg:0.603	std:0.052
	score_time - 		avg:0.144	std:0.001
	test_accuracy - 	avg:0.537	std:0.009
	test_precision - 	avg:0.538	std:0.016
	test_recall - 		avg:0.561	std:0.084
	test_f1 - 		avg:0.545	std:0.034
Col: quotation_cleanC
X shape: (10000,)
	fit_time - 		avg:0.366	std:0.025
	score_time - 		avg:0.083	std:0.005
	test_accuracy - 	avg:0.535	std:0.009
	test_precision - 	avg:0.529	std:0.007
	test_recall - 		avg:0.640	std:0.004
	test_f1 - 		avg:0.579	std:0.006
Col: quotation_cleanD
X shape: (10000,)
	fit_time - 		avg:0.335	std:0.011
	score_time - 		avg:0.078	std:0.005
	test_accuracy - 	avg:0.529	std:0.006
	test_precision - 	avg:0.527	std:0.009
	test_recall - 		avg:0.575	std:0.063
	test

In [70]:
pipeline = Pipeline([
            ('vect', TfidfVectorizer(ngram_range=(1,3), max_features=1_000_000)),
            ('clf', XGBClassifier(use_label_encoder=False, eval_metric='error', n_jobs=4)), # logloss or error
        ])

test_classifer(pipeline, break_after_one_iter=True)

Col: quotation_cleanE
X shape: (260504,)
	fit_time - 		avg:86.983	std:6.430
	score_time - 		avg:4.505	std:0.376
	test_accuracy - 	avg:0.616	std:0.002
	test_precision - 	avg:0.655	std:0.003
	test_recall - 		avg:0.490	std:0.004
	test_f1 - 		avg:0.560	std:0.002


NICKY's CELLS

In [211]:
df_temp = df.sample(frac=1)
nicky_X = df_temp['quotation_cleanE'].values.copy()

nicky_y = y = df_temp['party_label'].values
nicky_y = np.array([0.0 if label=='R' else 1.0 for label in nicky_y])

In [212]:
nicky_pipeline = Pipeline([
            ('vect', TfidfVectorizer(ngram_range=(1,3), max_features=1_000_000)),
            ('clf', MultinomialNB()),
        ])

nicky_scoring=['accuracy', 'precision', 'recall', 'f1']
nicky_res = cross_validate(nicky_pipeline, nicky_X, nicky_y, scoring=nicky_scoring, cv=3)
{item: (value.mean(), value.std()) for (item, value) in nicky_res.items()}

{'fit_time': (21.35477900505066, 2.414792275802797),
 'score_time': (4.301192045211792, 0.20933518003005805),
 'test_accuracy': (0.7065687961470012, 0.0017286617663049687),
 'test_precision': (0.7105281368393941, 0.0025199203355492925),
 'test_recall': (0.6971869969344056, 0.0007832731861736124),
 'test_f1': (0.7037917553017575, 0.0012552082031326885)}

In [213]:
split_frac = 0.9

bound = round(len(nicky_X)*split_frac)

nicky_X_train = nicky_X[:bound]
nicky_y_train = nicky_y[:bound]

nicky_X_test = nicky_X[bound:]
nicky_y_test = nicky_y[bound:]


In [214]:
nicky_pipeline.fit(nicky_X_train, nicky_y_train)

In [ ]:
y_pred = nicky_pipeline.predict_proba(nicky_X_test)
y_pred.shape

(26050, 2)

In [ ]:
df_temp = df.sample(frac=1)
nicky_X = df_temp['quotation_cleanE'].values.copy()

nicky_y = y = df_temp['party_label'].values
nicky_y = np.array([0.0 if label=='R' else 1.0 for label in nicky_y])

In [ ]:
df.columns

Index(['quoteID', 'quotation', 'speaker', 'date', 'numOccurrences', 'id',
       'party_label', 'US_congress_bio_ID', 'quotation_cleanA',
       'quotation_cleanB', 'quotation_cleanC', 'quotation_cleanD',
       'quotation_cleanE'],
      dtype='object')

In [ ]:
nicky_df = df_temp.iloc[bound:].copy()
nicky_df['pred'] = y_pred[:,0]
nicky_df.head(2)

,quoteID,quotation,speaker,date,numOccurrences,id,party_label,US_congress_bio_ID,quotation_cleanA,quotation_cleanB,quotation_cleanC,quotation_cleanD,quotation_cleanE,pred
54552,2020-02-05-124571,"You will not change him. You cannot constrain him. He is who he is.... Now, do impartial justice and convict him.",adam schiff,2020-02-05 19:35:00,1,Q350843,D,S001150,"You will not change him. You cannot constrain him. He is who he is.... Now, do impartial justice and convict him.",you will not change him you cannot constrain him he is who he is now do impartial justice and convict him,change cannot constrain impartial justice convict,chang cannot constrain imparti justic convict,change can not constrain impartial justice convict,0.562421
129309,2020-02-02-031905,"Obviously, the analogy is not true,",bernie sanders,2020-02-02 21:57:00,2,Q359442,D,S000033,"Obviously, the analogy is not true,",obviously the analogy is not true,obviously analogy true,obvious analog true,obviously analogy true,0.506450


In [ ]:
nicky_df['rounded_pred'] = np.argmax(y_pred,axis = 1)

In [ ]:
import plotly.express as px
fig = px.scatter(nicky_df.sample(1000), x = 'party_label', y = 'pred', color = 'rounded_pred' ,hover_data= ['party_label','speaker','quotation'])
fig.show()

In [ ]:
def get_result_df(regressor,filename):
    
    results = []
    
    with pd.read_json(filename, lines=True, compression='bz2', chunksize=500) as df_reader:
        for df in df_reader:
            df = df.sample(50)
            x = df['glove_embedding'].values
            y = df[['party_number']].values[:,0]
            x = np.array(list(map(lambda val: np.array(val),x))).reshape(x.shape[0],-1)

            ypred = regressor.predict(x)

            tmp_df = pd.DataFrame(data = {'party_number': df['party_number'].values, \
                                        'prediction': ypred, 'quotation': df['quotation'].values, 'speaker': df['speaker'].values} )
            
            
            results.append(tmp_df.copy())
        
            result_df = pd.concat(results)
                
            result_df['rounded_pred'] = result_df['prediction'].apply(lambda x: 0 if x <= 0.5 else 1)
    display(result_df.head(1))
    
    fig = px.scatter(result_df, x = 'party_number', y = 'prediction', color = 'rounded_pred' ,hover_data= ['party_number','speaker','quotation'])
    fig.show()